<a href="https://colab.research.google.com/github/olodenm/practice/blob/main/wordcloudrendering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Скрипт для генирации "Облака слов" из выгрузки личной переписки с Telegram

## Как использовать этот код

1. **Получите свой JSON-файл**: Этот код предназначен для работы с JSON-файлами. Для этого вам нужно открыть Telegram desktop, зайти в переписку которую хотите выгрузить. Нажать на три точки в верхнем правом углу экрана, "экспорт истории чата" затем выбираем формат "Машиночитаемый JSON", путь можно указать любой удобный. выбрать диапазон дат, и нажать кнопку "Экспортировать"

2. **Загрузите JSON-файл в ту же директорию, что и код**: После того, как у вас есть JSON-файл, вы должны загрузить его в ту же директорию, что и ваш код. Это позволит коду легко найти и загрузить ваш JSON-файл.

3. **Убедитесь, что ваш JSON-файл имеет правильный формат**: Этот код ожидает, что ваш JSON-файл будет иметь определенный формат. В частности, он ожидает, что JSON-файл будет содержать ключ 'messages', который ссылается на список сообщений. Каждое сообщение должно быть словарем с ключом 'text', который ссылается на текст сообщения.

4. **Запустите код**: Теперь вы готовы запустить код! Если вы все сделали правильно, код должен успешно загрузить ваш JSON-файл, обработать текст сообщений и создать облако слов.

Если у вас возникнут вопросы или проблемы, не стесняйтесь обращаться!


In [13]:
import pandas as pd
import numpy as np
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from wordcloud import WordCloud
import matplotlib.pyplot as plt

In [ ]:
# Загружаем данные из JSON-файла
df = pd.read_json('result.json')

In [ ]:
# Вывести значения ключа 'text' из словарей в столбце 'messages'
texts = df['messages'].apply(lambda x: x.get('text') if x else None)

# Убрать строки, где ключ 'text' отсутствует
texts = texts.dropna()

In [ ]:
# Преобразовать Series в DataFrame
df = texts.to_frame()

In [ ]:
# Приводим весь текст к нижнему регистру;
df['messages'] = df['messages'].str.lower()

In [ ]:
# Удалить строки, в которых 'messages' пустой
df = df[df['messages'].notna() & (df['messages'] != '')]

In [ ]:
# уберем все служебные символы
df['messages'] = [re.sub('[\W_]+', ' ', text) for text in df['messages']]

In [ ]:
# Удаляем стоп-слова;
nltk.download('stopwords')
stopwords_set = set(stopwords.words('russian'))
df['messages'] = df['messages'].apply(lambda x: ' '.join([word for word in x.split() if word not in stopwords_set]))

In [ ]:
# Произведем лемматизацию
nltk.download('omw-1.4')
nltk.download('wordnet')
wordnet_lemmatizer = WordNetLemmatizer()
df['messages'] = df['messages'].apply(lambda x: ' '.join([wordnet_lemmatizer.lemmatize(word) for word in x.split() if word not in stopwords_set]))

In [ ]:
# Удалить строки, в которых 'messages' пустой
df = df[df['messages'].notna() & (df['messages'] != '')]

###Параметры функции WordCloud:
* **width**: Ширина изображения облака слов в пикселях.
* **height**: Высота изображения облака слов в пикселях.
* **background_color**: Цвет фона изображения облака слов. (представляет собой шестнадцатеричный код цвета.)
* **mode**: Режим цвета для изображения. "RGBA" означает красный, зеленый, синий и альфа (прозрачность).
* **collocations**: Если это значение установлено в True, то все биграммы (два слова, которые часто встречаются вместе) будут включены в облако слов.
* **max_words**: Максимальное количество слов, которые будут отображаться в облаке слов.
* **stopwords**: Набор стоп-слов, которые будут исключены из облака слов.


In [ ]:
# Объединить все строки в одну, разделенную пробелами
text = ' '.join(df['messages'].dropna())

# Обновить список стоп-слов, включая "ещё"
stopwords_set.update(['ещё', 'это'])

# Создание облака слов
wordcloud = WordCloud(width=1000, height=1000, background_color='#141613', mode='RGBA', collocations=True, max_words=100, stopwords=stopwords_set)
wordcloud.generate(text)

# Визуализация облака слов
plt.figure(figsize=(20,10)) # Размер изображения
plt.imshow(wordcloud, interpolation='bicubic')
plt.axis("off")
plt.margins(x=0, y=0)
plt.show()

In [28]:
# Сохранение изображения
wordcloud.to_file("worldcloud.png")